In [69]:
import json
import pandas as pd
from collections import Counter
import numpy as np
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import fcluster
from sklearn.cluster import MiniBatchKMeans
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.metrics.pairwise import euclidean_distances

In [4]:
albums_json = json.load(open("1 Million Playlists/albums_0-999.json"))

artists_json = json.load(open("1 Million Playlists/artists_0-999.json"))
artists_pd = pd.read_csv("CSV/artists.csv")

songs_json = json.load(open("1 Million Playlists/songs_0-999.json"))
songs_pd = pd.read_csv("CSV/songs.csv")

playlists = json.load(open("1 Million Playlists/mpd.slice.0-999.json"))

vetting_json = json.load(open("Vetting_Playlist.json"))
vetting_pd = pd.read_csv("Vetting_Playlist.json")

In [5]:
def artist_counter(playlist):
    artists = []
    for i in playlist['tracks']['items']:
        for j in i['track']['artists']:
            artists.append(j['name'])
    artist_counter = Counter(artists)
    sorted_artists = sorted(artist_counter, key=artist_counter.get, reverse=True)
    return sorted_artists

vetting_artists = artist_counter(vetting_json)

In [39]:
vetting_songs = []
for i in vetting_json['tracks']['items']:
    vetting_songs.append(i['track']['name'])

In [6]:
def artist_counter_2(playlist): #Vetting playlist is structured differently to other playlists so this new function helps finds the artist for the other playlists
    artists = []
    for i in playlist["tracks"]:
        artists.append(i["artist_name"])
    artist_counter = Counter(artists)
    sorted_artists = sorted(artist_counter, key=artist_counter.get, reverse=True)
    return sorted_artists

In [7]:
playlists_list = playlists["playlists"]

In [15]:
linked_list = []
for i in playlists_list:
    array = artist_counter_2(i)
    linked_list.append(array)

In [20]:
# Define a function to calculate similarity between two lists
def similarity(list1, list2):
    """
    Calculate Jaccard similarity between two lists.
    """
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

# Define a function to find the most similar list
def find_most_similar_list(main_list, linked_list):
    """
    Find the most similar list in the linked list to the main list.
    """
    max_similarity = 0
    most_similar_list = None
    most_similar_index = None
    for index, sublist in enumerate(linked_list):
        similarity_score = similarity(main_list, sublist)
        if similarity_score > max_similarity:
            max_similarity = similarity_score
            most_similar_list = sublist
            most_similar_index = index
    return most_similar_list, max_similarity, most_similar_index


# Find the most similar list
most_similar, similarity_score, index = find_most_similar_list(vetting_artists, linked_list)
print("Most similar list:", most_similar)
print("Similarity score:", similarity_score)
print("Index of most similar list:", index)


Most similar list: ['Jack Johnson', 'Vampire Weekend', 'The Strokes', 'Local Natives', 'Sublime', 'Paul Simon', 'OK Go', 'Clap Your Hands Say Yeah', 'Dr. Dog', 'Jonathan Edwards', 'MGMT', 'The Morning Benders', 'Surfer Blood', 'Okkervil River', 'Two Door Cinema Club', 'Delta Spirit', 'Faces', 'Wilco', 'Daryl Hall & John Oates', 'Coconut Records', 'Spoon', 'Someone Still Loves You Boris Yeltsin', 'Queen', 'Beck']
Similarity score: 0.08045977011494253
Index of most similar list: 249


In [22]:
best_matching_playlist = playlists_list[249]

In [58]:
best_matching_playlist_artists = []
best_matching_playlist_songs = []
best_matching_playlist_songs_id = []
for i in best_matching_playlist["tracks"]:
    best_matching_playlist_artists.append(i["artist_name"])
    best_matching_playlist_songs.append(i["track_name"])
    best_matching_playlist_songs_id.append(i["track_uri"][14:])
    

In [84]:
# Find common elements using list comprehension
Best_Artist_Match = [item for item in vetting_artists if item in best_matching_playlist_artists]

# Print the third list
print(f'Best Artist Match: {Best_Artist_Match}')

Best Artist Match: ['Dr. Dog', 'Delta Spirit', 'Local Natives', 'Spoon', 'The Strokes', 'Vampire Weekend', 'Clap Your Hands Say Yeah']


Best Match for Heirchial Clustering: Heart It Races - Dr. Dog Version